In [1]:
import sys
import pandas as pd
from geopy.geocoders import Nominatim
import csv

In [2]:
def replace_duplicate_locations(df_antennas,df_cdr):
    print(df_antennas)
    print("====")
    antennas_lat = df_antennas.LAT
    print(antennas_lat)
    print("====")
    antennas_long = df_antennas.LONG
    print(antennas_long)
    print("====")
    
    unique_lat_long = list(set(tuple(zip(antennas_lat,antennas_long))))
    
    print(unique_lat_long)
    
    
    for location in unique_lat_long:
        primary_antenna = -10
        secondary_antennas = []
        for row in df_antennas.itertuples():
            if row.LAT == location[0] and row.LONG == location[1]:
                if primary_antenna == -10:
                    primary_antenna = row.CELLID
                else:
                    #row.CELLID = primary_antenna
                    secondary_antennas.append(row.CELLID)
                    df_antennas.at[row[0],'CELLID'] = primary_antenna
            #end
        #end
        
        ### Aqui substituir no df_cdr
        
        #print(df_cdr.head(2))
        #print(primary_antenna)
        #print(secondary_antennas)

        df_cdr['antenna'] = df_cdr['antenna'].replace(secondary_antennas,primary_antenna)
        #exit()
    #end
    df_antennas = df_antennas.drop_duplicates()
    #print(len(set(df_cdr.antenna)))
    return [df_antennas,df_cdr]
#end

In [3]:
def read_df_from_file(filename):
	df = pd.read_csv(filename, sep=";")
	return df

In [5]:
# ***** Depois da leitura do CDR, eu tinha feito um passo (que pode não ser obrigatório)   *****
# ***** e que envolve remover as antenas com localização duplicada (no arquivo de antenas) *****
# ***** e substituir as antenas pelas antenas com localizações únicas no CDR.              *****

cdr_filename_region = "/media/vinicius/vinicius-HDD3TB/TODAS_UFS_CDR/jf/cdr_regiao_imediata_jf_all.txt"
df_cdr = read_df_from_file(cdr_filename_region)
antenna_filename_region = "/media/vinicius/vinicius-HDD3TB/TODAS_UFS_CDR/jf/antennas_jf.txt"
df_antennas = read_df_from_file(antenna_filename_region)

print(df_cdr.head(20))
print(df_antennas)

          date      time  duration  ddd_from  \
0   2013-03-21  18:55:11      1.63        32   
1   2013-03-21  19:45:36      0.07        32   
2   2013-03-21  13:56:03     14.58        32   
3   2013-03-21  14:55:45      2.98        83   
4   2013-03-21  19:08:59      0.08        83   
5   2013-03-21  15:02:06      0.08        83   
6   2013-03-21  18:14:42      0.40        83   
7   2013-03-21  08:10:29      0.75        32   
8   2013-03-21  17:47:14      5.10        83   
9   2013-03-21  14:15:24      1.27        32   
10  2013-03-21  18:13:04      0.07        32   
11  2013-03-21  16:31:10      0.95        83   
12  2013-03-21  13:25:36      0.15        83   
13  2013-03-21  13:07:02      0.30        32   
14  2013-03-21  18:51:26      0.22        83   
15  2013-03-21  04:43:07      0.57        61   
16  2013-03-21  20:15:51      2.43        32   
17  2013-03-21  19:03:25      0.38        83   
18  2013-03-21  07:03:40      0.43        83   
19  2013-03-21  10:47:38      7.55      

In [6]:
[df_antennas,df_cdr] = replace_duplicate_locations(df_antennas,df_cdr)

print(df_antennas)
print(df_cdr.head(20))

df_antennas.to_csv (r"/media/vinicius/vinicius-HDD3TB/TODAS_UFS_CDR/jf/antennas_jf_unique.txt", index = False, header=True, sep=";")
df_cdr.to_csv (r'/media/vinicius/vinicius-HDD3TB/TODAS_UFS_CDR/jf/cdr_regiao_imediata_jf_all_unique.txt', index = False, header=True, sep=";")

     CELLID  UF  DDD       LAT      LONG            CITY  \
0     17178  MA   98  -3.60925 -45.34292   Santos Dumont   
1     29922  MG   32 -21.77560 -43.43085    Juiz de Fora   
2     29953  MG   32 -21.79245 -43.38823    Juiz de Fora   
3     23733  MG   32 -21.78109 -43.36022    Juiz de Fora   
4     51492  MG   32 -21.72198 -43.40184    Juiz de Fora   
..      ...  ..  ...       ...       ...             ...   
341   50791  MG   32 -21.66369 -43.43173    Juiz de Fora   
342   50882  MG   32 -21.78162 -43.34295    Juiz de Fora   
343   50883  MG   32 -21.78162 -43.34295    Juiz de Fora   
344   65771  MG   32 -21.67006 -43.43436    Juiz de Fora   
345   50613  MG   32 -21.87465 -43.33599  Matias Barbosa   

                  MUNICIPALITY  
0      Microrregião de Pindaré  
1    Microrregião Juiz de Fora  
2    Microrregião Juiz de Fora  
3    Microrregião Juiz de Fora  
4    Microrregião Juiz de Fora  
..                         ...  
341  Microrregião Juiz de Fora  
342  Microrregi